In [1]:
from IPython.display import display, HTML
display(HTML("""
<style>
div.container{width:99% !important;}
div.cell.code_cell.rendered{width:100%;}
div.input_prompt{padding:0px;}
div.CodeMirror {font-family:Consolas; font-size:24pt;}
div.text_cell_render.rendered_html{font-size:20pt;}
div.text_cell_render li, div.text_cell_render p, code{font-size:22pt; line-height:40px;}
div.output {font-size:24pt; font-weight:bold;}
div.input {font-family:Consolas; font-size:24pt;}
div.prompt {min-width:70px;}
div#toc-wrapper{padding-top:120px;}
div.text_cell_render ul li{font-size:24pt;padding:5px;}
table.dataframe{font-size:24px;}
</style>
"""))

# <span style="color:rgb(0,200,100);">1. 패키지 </span>

In [5]:
# python-dotenv, langchain(1.2.0),  langchain-openai,  langchain-pinecone,
# pandas,  langchain-community, docx2txt,  langchain-text_splitters,  langchain_ollama

# <span style="color:rgb(0,200,100);">2. 환경설정(환경변수, 시스템파라미터변수) </span>

In [2]:
import os
from dotenv import load_dotenv
load_dotenv()
OPENAI_API_KEY = os.getenv("OPENAI_API_KEY")
PINECONE_API_KEY = os.getenv("PINECONE_API_KEY")
OPENAI_LLM_MODEL = "gpt-4o-mini"
OPENAI_EMBEDDING_MODEL = "text-embedding-3-large" # 차원수 3072

PINECONE_INDEX_NAME = "better-rag-index"
PINECONE_INDEX_DEMENSION = 3072
PINECONE_INDEX_METRIC = "cosine"
PINECONE_INDEX_REGION = "us-east-1"
PINECONE_INDEX_CLOUD = "aws"

# <span style="color:rgb(0,200,100);">3. 문서를 chunk로 분할하기 </span>

In [3]:
from langchain_community.document_loaders import Docx2txtLoader
from langchain_text_splitters import RecursiveCharacterTextSplitter
loader = Docx2txtLoader('./data/소득세법_with_markdown.docx')
document = loader.load()
text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=1500,
    chunk_overlap=200, 
    separators=["\n\n", "\n", " ", ""]
)
# documents = loader.load_and_split(text_splitter)
documents = text_splitter.split_documents(document)
print(f"총 {len(documents)}개 청크 생성")

총 194개 청크 생성


# <span style="color:rgb(0,200,100);">4. metadata 추가하기 </span>
- 청크 내용의 카테고리, 청크내용의 title, 조항

In [4]:
import re
def remove_special_chars(text:str) -> str:
    "특수문자 및 \n제거(:는 그대로)"
    # \n제거
    text = text.replace("\n", " ")
    # 한글, 영문, 숫자, 공백, 마침표, 콤마, :만 남기고 전부 제거
    cleaned = re.sub(r'[^가-힣a-zA-Z0-9\s,\.:]', '', text)
    # 불필요한 중복 공백을 제거
    cleaned = re.sub(r'\s+', ' ', cleaned).strip()
    return cleaned
# 사용예시
content = "**소득세 납세 의무 **: \n\n이 법 또는 「법인세법」에 따라 소득에 대한 소득세를 부과"
print(remove_special_chars(content))

소득세 납세 의무 : 이 법 또는 법인세법에 따라 소득에 대한 소득세를 부과


In [18]:
# 제목을 추출하는 함수(exaone3.5) : powershell이나 cmd창에서 ollama pull exaone3.5:2.4b
from langchain_ollama import ChatOllama
def extract_title_with_llm(content):
    "exaone3.5를 사용하여 content의 제목을 추출"
    title_extractor_llm = ChatOllama(
        model = "exaone3.5:2.4b",
        temperature = 0.1,
        num_predict = 100 # 최대 토큰 30토큰까지만 출력
    )
    prompt = f"""다음 소득세법 조문의 핵심 제목을 100토큰 이내로 완벽하게 말이 되도록 추출해 주세요.
    중간에 말이 끊기면 안 되요.
    조문 : {content}"""
    ai_message = title_extractor_llm.invoke(prompt)
    title = ai_message.content.strip()
    return remove_special_chars(title)
content = """1. 원천징수하는 자가 거주자인 경우: 그 거주자의 주된 사업장 소재지. 다만, 주된 사업장 외의 사업장에서 원천징수를 하는 경우에는 그 사업장의 소재지, 사업장이 없는 경우에는 그 거주자의 주소지 또는 거소지로 한다.
2. 원천징수하는 자가 비거주자인 경우: 그 비거주자의 주된 국내사업장 소재지. 다만, 주된 국내사업장 외의 국내사업장에서 원천징수를 하는 경우에는 그 국내사업장의 소재지, 국내사업장이 없는 경우에는 그 비거주자의 거류지(居留地) 또는 체류지로 한다.
3. 원천징수하는 자가 법인인 경우: 그 법인의 본점 또는 주사무소의 소재지
"""
print(extract_title_with_llm(content))

소득세 원천징수 기준: 거주자는 주된 사업장 소재지, 비거주자는 주된 국내 사업장 소재지 적용. 사업장 부재 시 주소지 또는 거류지로 지정. 법인은 본점 또는 주사무소 소재지 기준.


In [19]:
def categorize_content(content):
    "내용 카테고리 분류"
    # 쳇 gpt가 meta 데이터에 쓴다고 하니 만들어 준 카테고리
    categories = {
    '납세의무': ['납세의무', '거주자', '비거주자', '원천징수', '공동사업자', '상속인', '증여자', '신탁재산'],
    '세율계산': ['세율', '과세표준', '산출세액', '결정세액', '종합소득', '퇴직소득'],
    '근로소득': ['근로소득', '총급여', '급여', '연봉', '임금', '퇴직소득'],
    '사업소득': ['사업소득', '공동사업', '주택임대소득', '부업소득'],
    '이자배당': ['이자소득', '배당소득', '예금이자', '채권', '의제배당'],
    '양도소득': ['양도소득', '자산양도', '부동산양도', '주식양도'],
    '연금소득': ['연금소득', '공적연금', '사적연금', '연금보험'],
    '기타소득': ['기타소득', '상금', '보상금', '발명보상금', '종교인소득'],
    '공제감면': ['공제', '소득공제', '세액공제', '기본공제', '감면'],
    '비과세': ['비과세', '면제', '복무급여', '실업급여', '출산휴가급여', '장학금'],
    '신고납부': ['신고', '납부', '납세지', '신고기한', '원천징수'],
    '과세기간': ['과세기간', '과세연도', '사업연도'],
    '과세소득구분': ['종합소득', '퇴직소득', '양도소득', '금융투자소득'],
    }
    result = [] # 각 카테고리 추가
    for category, keywords in categories.items():
        #print(category, keywords)
        #print(any([keyword in content for keyword in keywords]))
        if any([keyword in content for keyword in keywords]):
            result.append(category)
    if not result:
        result.append("기타")
    return result
categorize_content(content)

['납세의무', '신고납부']

- 개선된 카테고리 분류방법 : 키워드의 중요도에 따라 가중치를 부여하고, 가중치가 높은 순으로 카테고리 반환

In [7]:
def get_category():
    return {
        '납세의무': {
            '납세의무': 3, '거주자': 3, '비거주자': 3, '납세의무자': 3, 
            '원천징수': 2, '원천징수의무자': 2, '공동사업자': 2, 
            '상속인': 2, '증여자': 2, '신탁재산': 2
        },

        '세율계산': {
            '세율': 3, '소득세': 3, '과세표준': 3, '산출세액': 3, '세액': 3,
            '결정세액': 2, '세액계산': 2, '기본세율': 2, '세율적용': 2,
            '누진세율': 2, '종합소득세': 2
        },

        '근로소득': {
            '근로소득': 3, '총급여': 3, '급여': 3, '연봉': 3, '임금': 3,
            '근로소득금액': 2, '총급여액': 2, '상여': 2, '수당': 2,
            '봉급': 2, '직장인': 2
        },

        '사업소득': {
            '사업소득': 3, '총수입금액': 3, '필요경비': 3,
            '사업자': 2, '사업소득금액': 2, '결손금': 2, '이월결손금': 2,
            '주택임대소득': 2, '공동사업': 2
        },

        '이자배당': {
            '이자소득': 3, '배당소득': 3, '예금이자': 2, '채권': 2,
            '의제배당': 2, '배당세액공제': 2, '분리과세이자소득': 2,
            '분리과세배당소득': 2
        },

        '양도소득': {
            '양도소득': 3, '자산양도': 2, '부동산양도': 2, '주식양도': 2,
            '양도차익': 2, '취득가액': 2, '양도가액': 2, '양도소득금액': 2
        },

        '연금소득': {
            '연금소득': 3, '연금계좌': 3, '연금저축': 3,
            '퇴직연금': 2, '공적연금': 2, '사적연금': 2, '연금보험': 2,
            '연금수령': 2
        },

        '기타소득': {
            '기타소득': 3, '가상자산': 3, '가상자산소득': 2,
            '상금': 2, '보상금': 2, '종교인소득': 2, '원고료': 2,
            '복권': 1, '당첨금': 1, '발명보상금': 1
        },

        '공제감면': {
            '공제': 3, '소득공제': 3, '세액공제': 3,
            '기본공제': 2, '인적공제': 2, '특별공제': 2, '추가공제': 2,
            '근로소득공제': 2, '연금소득공제': 2, '퇴직소득공제': 2,
            '연금계좌세액공제': 2, '감면': 2
        },

        '비과세': {
            '비과세': 3, '비과세소득': 3, '면제': 2, '세액감면': 2,
            '소득세면제': 2, '복무급여': 1, '실업급여': 1, 
            '출산휴가급여': 1, '장학금': 1
        },

        '신고납부': {
            '신고': 3, '확정신고': 3, '과세표준확정신고': 3, 
            '납부': 3, '중간예납': 2, '가산세': 2,
            '신고기한': 2, '납부기한': 2, '납세지': 2
        },

        '과세기간': {
            '과세기간': 3, '과세연도': 3, '사업연도': 2, 
            '과세기간종료일': 2
        },

        '과세방식': {
            '종합과세': 3, '분리과세': 3, '합산과세': 2, 
            '종합소득과세표준': 2, '분리과세소득': 2, 
            '금융투자소득': 2
        },

        '장부기장': {
            '장부': 3, '복식부기': 3, '간편장부': 2, '기장': 2,
            '장부기록': 2, '증명서류': 2, '기장세액공제': 2
        }
    }

In [8]:
content = """1. 원천징수하는 자가 거주자인 경우: 그 거주자의 주된 사업장 소재지. 다만, 주된 사업장 외의 사업장에서 원천징수를 하는 경우에는 그 사업장의 소재지, 사업장이 없는 경우에는 그 거주자의 주소지 또는 거소지로 한다.
2. 원천징수하는 자가 비거주자인 경우: 그 비거주자의 주된 국내사업장 소재지. 다만, 주된 국내사업장 외의 국내사업장에서 원천징수를 하는 경우에는 그 국내사업장의 소재지, 국내사업장이 없는 경우에는 그 비거주자의 거류지(居留地) 또는 체류지로 한다.
3. 장부 장부 장부 장부 복식부기 복식부기 복식부기 복식부기 복식부기 복식부기
"""

In [9]:
def categorize_content(content, top_k=None):
    """내용 카테고리 분류 - 점수 기반으로 모든 카테고리를 점수 순으로 반환
    Parameters :
    - content : 분류할 텍스트 내용
    - top_k : 상위 몇개까지 카테고리를 반환할지(None이면 모든 카테고리 반환)
    Returns:
    - 카테고리 리스트(점수 높은 순)"""
    category_keywords = get_category()
    category_scores = {}
    # 각 카테고리별 점수 계산
    for category, weighted_keywords in category_keywords.items():
        # print(category, weighted_keywords)
        score = 0
        for keyword, weight in weighted_keywords.items():
            #if keyword in content: # content에 keyword 가 포함되어 있는지 여부
            #    score += weight
            count = content.count(keyword) # content에 keyword가 몇번 나오는지
            score += count * weight
        if score > 0:
            category_scores[category] = score
    # print(category_scores)
    # 내림차순 정렬한 카테고리 이름만 추출
    sorted_categories = sorted(category_scores.items(), key=lambda x:x[1], reverse=True) # 내림차순
    #print(sorted_categories)
    all_categories = [category[0] for category in sorted_categories]
    # print(all_categories)
    # 매칭되는 카테고리가 없으면 "기타" 반환
    if not all_categories:
        all_categories = ["기타"]
    # top_k가 지정되면 상위 top_k개만, 아니면 전체 반환
    if top_k is not None:
        return all_categories[:top_k]
    else:
        return all_categories
type(categorize_content(content))

list

In [10]:
categorize_content("연봉 5천만원인 회사원의 소득세는 얼마예요?")

['세율계산', '근로소득']

In [11]:
categorize_content(documents[46].page_content)

['세율계산', '공제감면', '이자배당', '납세의무', '과세방식', '과세기간', '사업소득']

In [12]:
# 해당 조항 추출하기
import re
def get_article(content):
    "조항들 추출"
    article_match = re.findall(r'제(\d+)조', content)
    article = list(set(article_match)) # 중복제거
    article.sort(key=lambda x:int(x))
    if article:
        return "「"+ ", ".join([f"{a}조" for a in article]) +"」"
print(get_article("무시기 무시기"))

None


In [15]:
# documents

In [94]:
%%time
# 메타데이터(source, title, category, article(조항))를 포함한 새로운 chunk
enhanced_chunks = []
for i, chunk in enumerate(documents):
    if i%20 == 0:
        print(f" 진행 중 : {i/len(documents)*100:.2f} % 진행")
    content = chunk.page_content
    metadata = chunk.metadata.copy()    
    metadata['title'] = extract_title_with_llm(content) # Exaone으로 제목 추출
    metadata['category'] = categorize_content(content) # 카테고리 추출
    metadata['chunk_id'] = f"chunk_{i:03d}"
    article = get_article(content)
    if article:
        metadata['article'] = article
    enhanced_chunks.append(type(chunk)(page_content=content, metadata=metadata))
print("확장된 chunk 처리 완료")

 진행 중 : 0.00 % 진행
 진행 중 : 10.31 % 진행
 진행 중 : 20.62 % 진행
 진행 중 : 30.93 % 진행
 진행 중 : 41.24 % 진행
 진행 중 : 51.55 % 진행
 진행 중 : 61.86 % 진행
 진행 중 : 72.16 % 진행
 진행 중 : 82.47 % 진행
 진행 중 : 92.78 % 진행
확장된 chunk 처리 완료
CPU times: total: 31.2 ms
Wall time: 34.9 ms


## <span style="color:rgb(0,100,200);">title추출에 시간이 너무 오래 걸릴 경우 </span>

In [20]:
import pandas as pd
df = pd.read_csv('./data/소득세법enhanced_chunks.csv', encoding="cp949")
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 194 entries, 0 to 193
Data columns (total 6 columns):
 #   Column        Non-Null Count  Dtype 
---  ------        --------------  ----- 
 0   page_content  194 non-null    object
 1   source        194 non-null    object
 2   title         194 non-null    object
 3   category      194 non-null    object
 4   chunk_id      194 non-null    object
 5   article       193 non-null    object
dtypes: object(6)
memory usage: 9.2+ KB


In [21]:
df.loc[2, 'category'], type(df.loc[2, 'category'])

("['납세의무', '이자배당', '양도소득', '근로소득', '사업소득', '연금소득', '기타소득', '신고납부', '과세기간']",
 str)

In [22]:
# 문자열 => 리스트
import ast
# 문자열
category = "['납세의무', '이자배당']"
# 리스트로 변환
category_list = ast.literal_eval(category)
print(category_list, type(category_list))

['납세의무', '이자배당'] <class 'list'>


In [23]:
df['category'].apply(lambda x : ast.literal_eval(x))
df['category'] = df['category'].apply(ast.literal_eval)
type(df.loc[0, 'category'])

list

In [24]:
import numpy as np
np.__version__

'2.2.6'

In [25]:
%%time
# 메타데이터(source, title, category, article(조항))를 포함한 새로운 chunk
from langchain_core.documents import Document

enhanced_chunks = [] # df내용을 Document list(enhanced_chunks)로 저장
for i, row in df.iterrows():
    content = row['page_content']
    source = row['source']
    title = row['title']
    category = row['category']
    chunk_id = row['chunk_id']
    article  = row['article']
    metadata = {"source":source, "title":title, "category":category, "chunk_id":chunk_id}
    if pd.notna(article): # article이 nan이 아니거나 None 아니거나?
        metadata['article'] = article
    # Document객체 생성 및 리스트 추가
    doc = Document(page_content=content, metadata=metadata)
    enhanced_chunks.append(doc)
print(len(enhanced_chunks) , '개 chunk 처리 완료')

194 개 chunk 처리 완료
CPU times: total: 15.6 ms
Wall time: 15.6 ms


# <span style="color:rgb(0,200,100);">5. 임베딩 모델 설정 </span>

In [26]:
from langchain_openai import OpenAIEmbeddings
embedding = OpenAIEmbeddings(
    model=OPENAI_EMBEDDING_MODEL,
    openai_api_key=OPENAI_API_KEY
)

# <span style="color:rgb(0,200,100);">6. Pinecone 인덱스 생성 및 vector store(DB) 저장 </span>

In [27]:
from pinecone import Pinecone, ServerlessSpec
from langchain_pinecone import PineconeVectorStore
# pinecone 클라언트
pc = Pinecone(api_key=PINECONE_API_KEY)
#print("pinecone index들 :", pc.list_indexes().names())
# 인덱스 생성 여부 확인 및 생성
#if PINECONE_INDEX_NAME not in pc.list_indexes().names():
if not pc.has_index(PINECONE_INDEX_NAME):
    pc.create_index(
        name=PINECONE_INDEX_NAME,
        dimension=PINECONE_INDEX_DEMENSION,
        metric=PINECONE_INDEX_METRIC,
        spec=ServerlessSpec(region=PINECONE_INDEX_REGION,
                           cloud=PINECONE_INDEX_CLOUD)
    )
    print('인덱스 생성 완료')
else:
    PineconeVectorStore.from_documents( # 인덱스에 값 하나 임의로 넣고 
        documents=[enhanced_chunks[0]],
        embedding=embedding,
        index_name = PINECONE_INDEX_NAME
    )
    index = pc.Index(PINECONE_INDEX_NAME)
    index.delete(delete_all=True) # index안의 데이터 삭제
    print(f"인덱스 {PINECONE_INDEX_NAME}이 이미 존재하여 데이터를 모두 지웠습니다")

인덱스 better-rag-index이 이미 존재하여 데이터를 모두 지웠습니다


In [28]:
# 빈 인덱스인지 확인
index.describe_index_stats()

{'dimension': 3072,
 'index_fullness': 0.0,
 'metric': 'cosine',
 'namespaces': {},
 'total_vector_count': 0,
 'vector_type': 'dense'}

In [29]:
%%time
# Pinecone 벡터 스토어 업로드
vector_database = PineconeVectorStore.from_documents(
    documents=enhanced_chunks,
    embedding=embedding,
    index_name = PINECONE_INDEX_NAME
)
print("벡터 DB 저장 완료")

벡터 DB 저장 완료
CPU times: total: 2.77 s
Wall time: 12.2 s


# <span style="color:rgb(0,200,100);">7. 관련문서 미리보기(metadata활용) </span>

In [30]:
query = "연봉 5000만원인 직장인의 소득세는 얼마예요?"
query = "종합소득 5000만원인 근로소득이 있는 거주자의 소득세는 얼마예요?"
categorize_content(query)

['납세의무', '세율계산', '근로소득', '과세소득구분']

In [31]:
# Retriever 생성 (category중 하나라도 포함되는 있는 chunk만)
retriever = vector_database.as_retriever(
    search_kwargs = {
        "k":3,
        "filter":{"category":{"$in":categorize_content(query)}}
    }
)
docs = retriever.invoke(query)
print("관련 문서")
for i, doc in enumerate(docs):
    category = doc.metadata['category']
    article  = doc.metadata.get('article', '조항없음')
    chunk_id = doc.metadata.get('chunk_id')
    title = doc.metadata.get('title')
    print(f"{i+1}.{chunk_id}-{article} {category} - {title}")

관련 문서
1.chunk_046-「14조, 17조, 55조, 56조」 ['세율계산', '이자배당', '공제감면', '과세방식', '납세의무', '사업소득', '과세기간'] - 제55조세율 거주자의 종합소득에 대한 소득세 세율 및 퇴직소득 산출세액 계산 방법.
2.chunk_038-「4조, 16조, 45조, 46조, 47조, 133조, 156조」 ['공제감면', '근로소득', '세율계산', '신고납부', '납세의무', '이자배당', '사업소득', '연금소득', '과세기간', '과세방식'] - 소득세법 제46조 및 제46조의2의 핵심 제목: 채권 등에 대한 소득금액의 계산 특례 및 중도 해지로 인한 이자소득금액 계산의 특례
3.chunk_005-「6조, 9조, 10조, 11조, 12조, 99조」 ['납세의무', '사업소득', '세율계산', '근로소득', '비과세', '신고납부', '과세기간'] - 소득세법 제10조납세지의 변경신고 및 제11조과세 관할와 제12조비과세소득 관련 규정 요약.


In [32]:
categorize_content(query)

['납세의무', '세율계산', '근로소득']

In [35]:
[{"category": {"$eq":category} } for category in categorize_content(query)]

[{'category': {'$eq': '납세의무'}},
 {'category': {'$eq': '세율계산'}},
 {'category': {'$eq': '근로소득'}}]

In [37]:
# # Retriever 생성 (category 3개가 모두 포함된 chunk)
filter_condition = {
    "$and" : [{"category": {"$eq":category} } for category in categorize_content(query)]
}
retriever = vector_database.as_retriever(
    search_kwargs = {
        "k":4,
        "filter":filter_condition
    }
)
docs = retriever.invoke(query)
print("관련 문서 : 납세의무, 세율계산, 근로소득")
for i, doc in enumerate(docs):
    chunk_id = doc.metadata.get("chunk_id")
    category = doc.metadata.get("category")
    title = doc.metadata.get("title")
    print(f"{i+1}. {chunk_id} - {category} : {title}")

관련 문서 : 납세의무, 세율계산, 근로소득
1. chunk_038 - ['공제감면', '근로소득', '세율계산', '신고납부', '납세의무', '이자배당', '사업소득', '연금소득', '과세기간', '과세방식'] : 소득세법 제46조 및 제46조의2의 핵심 제목: 채권 등에 대한 소득금액의 계산 특례 및 중도 해지로 인한 이자소득금액 계산의 특례
2. chunk_005 - ['납세의무', '사업소득', '세율계산', '근로소득', '비과세', '신고납부', '과세기간'] : 소득세법 제10조납세지의 변경신고 및 제11조과세 관할와 제12조비과세소득 관련 규정 요약.
3. chunk_051 - ['근로소득', '세율계산', '공제감면', '사업소득', '납세의무', '과세기간'] : 제58조재해손실세액공제 및 제59조근로소득세액공제
4. chunk_009 - ['세율계산', '근로소득', '이자배당', '공제감면', '납세의무', '사업소득', '기타소득', '연금소득', '과세기간', '과세방식'] : 소득세법 제14조: 과세표준의 계산 및 종합소득과세표준의 정의, 소득의 합산 제외 항목.


# <span style="color:rgb(0,200,100);">8. langchain 답변 생성 </span>

In [38]:
from langchain_openai import ChatOpenAI, OpenAIEmbeddings
from langchain_chroma import Chroma
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnablePassthrough
from dotenv import load_dotenv

In [39]:
# 1. LLM과 임베딩 초기화
load_dotenv()
llm = ChatOpenAI(model = OPENAI_LLM_MODEL)
embedding = OpenAIEmbeddings(model=OPENAI_EMBEDDING_MODEL)
# 2. 업로드한 벡터 db를 가져올 때
vector_database = PineconeVectorStore(
    embedding=embedding,
    index_name=PINECONE_INDEX_NAME
)

In [42]:
# 3. 키워드 사전 활용
keyword_dict  = [
    "사람을 나타내는 표현 -> 거주자",
    "직장인 -> 근로소득이 있는 거주자", 
    "월급쟁이 -> 근로소득이 있는 거주자",
    "회사원 -> 근로소득이 있는 거주자",
    "연봉 -> 종합소득",
    "월급 -> 근로소득",
    "세금 -> 소득세",
    "공제받다 -> 공제를 적용받다",
    "얼마나 내야하나 -> 세액은 얼마인가",
    "계산해줘 -> 계산하면 얼마인가"
]
prompt = ChatPromptTemplate.from_template(f"""사용자의 질문을 보고, 우리의 사전을 참고해서 
사용자의 질문을 변경해 주세요. 만약 변경할 필요가 없을경우, 사용자의 질문을 변경하지 않아도 됩니다.
그런 경우에는 질문만 리턴해 주세요.
사전 : {keyword_dict}
질문 : {{question}}""")
# llm.invoke(prompt.invoke("연봉 5000만원인 직장인의 소득세는 얼마예요?"))
keyword_chain = prompt | llm | StrOutputParser()
keyword_chain.invoke({"question":"연봉 5000만원인 직장인의 소득세는 얼마예요?"})

'종합소득 5000만원인 근로소득이 있는 거주자의 소득세는 세액은 얼마인가?'

In [43]:
keyword_chain.invoke("월급이 높은 남자가 있습니다")

'근로소득이 있는 거주자가 있습니다.'

In [45]:
# 3. Retriever 생성(category중 하나라도 포함되는 있는 chunk만)
retriever = vector_database.as_retriever(
    search_kwargs = {
        "k":3,
        "filter":{"category":{"$in":categorize_content(query)}}
    }
)
# 4. 프롬프트 템플릿
template = f"""당신은 최고의 한국 소득세 전문가입니다.
다음 문맥을 참고하여 질문에 답하세요.
답을 모르면 모른다고 답하세요.
최대 3문장으로 간결하게 답변하세요.
질문 : {{query}}
문맥 : {{context}}
답변 : """
prompt = ChatPromptTemplate.from_template(template)
# 5. 검색된 document를 텍스트로 변환하는 함수
def format_documents(documents):
    return  "\n\n---\n\n".join([doc.page_content for doc in documents])
# 6. RAG 체인 구성 (LCEL 방식)
from langchain_core.runnables import RunnablePassthrough # {"query":"~"} => "~"
rag_chain = (
    {
        "context":retriever | format_documents,
        "query":RunnablePassthrough() # 질문 그대로 전달
    }
    | prompt # prompt에 context와 query 변수 주입
    | llm    
    | StrOutputParser()
)
final_chain = keyword_chain | rag_chain
# 7. 실행
query = "연봉 5천만원인 직장인의 소득세는 얼마인가요?"
result = final_chain.invoke(query)
print(result + "\n*위의 답변은 AI에 의해 생성된 답변이므로 약간의 오차가 발생할 수 있습니다*")

종합소득 5천만원인 근로소득이 있는 거주자의 소득세는 84만원 + (1,400만원을 초과하는 금액 3,600만원의 15%)로 계산됩니다. 따라서, 소득세는 84만원 + 540만원 = 624만원입니다. 최종적으로, 해당 근로소득자의 소득세는 624만원입니다.
*위의 답변은 AI에 의해 생성된 답변이므로 약간의 오차가 발생할 수 있습니다*


# <span style="color:rgb(0,200,100);">9. 참조 조항 포함한 답변 생성 </span>

In [47]:
# 3. Retriever 생성(category중 하나라도 포함되는 있는 chunk만)
query = "종합소득 5000만원인 근로소득이 있는 거주자의 소득세는 세액은 얼마인가?"
retriever = vector_database.as_retriever(
    search_kwargs = {
        "k":3,
        "filter":{"category":{"$in":categorize_content(query)}}
    }
)
documents = retriever.invoke(query)

In [60]:
# 참조조항을 추출하는 함수
def extract_articles_from_docs(documents):
    """검색된 문서들(documents)에서 article정보를 추출하여 리스로 변환
    Returns : 중복이 제거된 조항 리스트 ex : ['제4조', '제16조']
    """
    articles = []
    for doc in documents:
        article = doc.metadata.get('article', '조항없음')
        # 「」제거. 문자를 list로 분리
        if article != '조항없음':
            article = article.replace("「", "").replace("」", "")
            article_list = article.split(", ")
            articles.extend(article_list)
    # 중복 제거 및 정렬
    unique_articles = list(set(articles))
    unique_articles.sort(key=lambda x : int(x[:-1]))
    final_articles = ["제"+article for article in unique_articles]
    return "소득세법 " + ",".join(final_articles)
extract_articles_from_docs(documents)

'소득세법 제4조,제6조,제9조,제10조,제11조,제12조,제14조,제16조,제17조,제45조,제46조,제47조,제55조,제56조,제99조,제133조,제156조'

In [ ]:
# RAG 체인 구성 - 참조조항을 포함한 답변 출력

In [63]:
# 1. 질문을 표준화
query = "연봉 5천만원인 직장인의 소득세는 얼마인가요?" # 출력함
normalized_query = keyword_chain.invoke({"question":query}) # 출력함
# 2. 관련문서 검색
retrieved_docs = retriever.invoke(normalized_query)
referenced_articles = extract_articles_from_docs(retrieved_docs) # 참조조항 출력

# 프롬프트 템플릿(위에것 그대로)을 이용한 rag 체임 구성
template = f"""당신은 최고의 한국 소득세 전문가입니다.
다음 문맥을 참고하여 질문에 답하세요.
답을 모르면 모른다고 답하세요.
최대 3문장으로 간결하게 답변하세요.
질문 : {{query}}
문맥 : {{context}}
답변 : """
prompt = ChatPromptTemplate.from_template(template)
prompt_chain = prompt | llm | StrOutputParser()
result = prompt_chain.invoke({
    "context":format_documents(retrieved_docs), 
    "query":query
})

In [68]:
print("원본 질문 :", {query})
print("표준화된 질문 :", {normalized_query})
print("="*70)
print("\n✅ 답변 :", result)
print("\n📌 참조조항 :", referenced_articles)
print("\n* 위 답변은 AI에 의해 생성된 답변이므로 약간의 오차가 날 수 있습니다 *")

원본 질문 : {'연봉 5천만원인 직장인의 소득세는 얼마인가요?'}
표준화된 질문 : {'연봉 5천만원인 근로소득이 있는 거주자의 소득세는 얼마인가요?'}

✅ 답변 : 연봉 5천만원인 직장인의 소득세는 84만원에 1,400만원을 초과하는 금액의 15%를 더한 금액으로 계산됩니다. 즉, 5천만원에서 1,400만원을 빼면 3,600만원이 남으며, 이 금액에 15%를 적용하면 540만원이 추가됩니다. 따라서 총 소득세는 84만원 + 540만원 = 624만원입니다.

📌 참조조항 : 소득세법 제4조,제6조,제9조,제10조,제11조,제12조,제14조,제16조,제17조,제45조,제46조,제47조,제55조,제56조,제99조,제133조,제156조

* 위 답변은 AI에 의해 생성된 답변이므로 약간의 오차가 날 수 있습니다 *


In [70]:
# 자사용 가능한 함수로 만들기
def ask_with_reference(query:str, k:int=4):
    "query, 표준화query, 답변, 참조사항 출력"
    # llm, embedding, vector_database, keyword_chain
    # 1. 질문을 표준화
    normalized_query = keyword_chain.invoke({"question":query}) # 출력함
    # 2. 관련문서 검색
    retriever = vector_database.as_retriever(
        search_kwargs = {
            "k":k,
            "filter":{"category":{"$in":categorize_content(query)}}
        }
    )
    # 관련문서 rerank
    retrieved_docs = retriever.invoke(normalized_query)
    referenced_articles = extract_articles_from_docs(retrieved_docs) # 참조조항 출력

    # 프롬프트 템플릿(위에것 그대로)을 이용한 rag 체임 구성
    template = f"""당신은 최고의 한국 소득세 전문가입니다.
    다음 문맥을 참고하여 질문에 답하세요.
    답을 모르면 모른다고 답하세요.
    최대 3문장으로 간결하게 답변하세요.
    질문 : {{query}}
    문맥 : {{context}}
    답변 : """
    prompt = ChatPromptTemplate.from_template(template)
    prompt_chain = prompt | llm | StrOutputParser()
    result = prompt_chain.invoke({
        "context":format_documents(retrieved_docs), 
        "query":query
    })
    print("원본 질문 :", {query})
    print("표준화된 질문 :", {normalized_query})
    print("="*70)
    print("\n✅ 답변 :", result)
    print("\n📌 참조조항 :", referenced_articles)
    print("\n* 위 답변은 AI에 의해 생성된 답변이므로 약간의 오차가 날 수 있습니다 *")

In [71]:
# 사용 예시
ask_with_reference("연봉 5천만원인 직장인 소득세는?")

원본 질문 : {'연봉 5천만원인 직장인 소득세는?'}
표준화된 질문 : {'종합소득 5천만원인 근로소득이 있는 거주자 소득세는?'}

✅ 답변 : 연봉 5천만원인 직장인의 소득세는 84만원에 1,400만원을 초과하는 금액에 대해 15%를 적용하여 계산합니다. 구체적으로, 5천만원에서 1,400만원을 빼면 3,600만원이고, 이 금액의 15%는 540만원입니다. 따라서 소득세는 84만원 + 540만원 = 624만원이 됩니다.

📌 참조조항 : 소득세법 제4조,제6조,제9조,제10조,제11조,제12조,제14조,제16조,제17조,제22조,제45조,제46조,제47조,제48조,제49조,제55조,제56조,제99조,제133조,제156조

* 위 답변은 AI에 의해 생성된 답변이므로 약간의 오차가 날 수 있습니다 *


In [72]:
# 사용 예시
ask_with_reference("양도 소득세는 얼마인가요?")

원본 질문 : {'양도 소득세는 얼마인가요?'}
표준화된 질문 : {'양도 소득세는 얼마인가요?'}

✅ 답변 : 양도소득세는 거주자의 양도소득 과세표준에 따라 세율을 적용하여 계산됩니다. 세율은 자산의 종류 및 보유 기간에 따라 다르며, 일반적으로 6%부터 시작하여 최대 45%까지 적용될 수 있습니다. 구체적인 세율과 계산 방식은 양도소득에 대한 법률에 명시되어 있습니다.

📌 참조조항 : 소득세법 제14조,제17조,제47조,제55조,제56조,제90조,제93조,제94조,제95조,제97조,제101조,제102조,제103조,제104조,제114조,제115조,제283조,제286조,제360조

* 위 답변은 AI에 의해 생성된 답변이므로 약간의 오차가 날 수 있습니다 *
